# Get a summary of training status of all alive subjects in a lab

In [1]:
import datajoint as dj
from ibl_pipeline import subject, action, acquisition, behavior
from ibl_pipeline.analyses import behavior as behavior_analyses
import re

Connecting zoe.ashwood@datajoint.internationalbrainlab.org:3306


## Get all alive subjects in a lab with water administration and weight

In [2]:
subjects = (subject.SubjectLab * subject.Subject  - subject.Death) & 'lab_name = "churchlandlab"' \
    & 'sex!="U"' & action.Weighing & action.WaterAdministration

In [3]:
subjects

subject_uuid,lab_name name of lab,subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_source name of source,protocol_number protocol number,subject_description
0936a724-2151-446f-98b2-92c3e9dd36dc,churchlandlab,CSHL_007,F,2018-11-27,L,C57BL/6J,Jax,3,None
1208c089-8b8e-4a87-98f0-05a68fb18370,churchlandlab,IBL_13,M,2018-04-23,None,None,Charles River,1,None
19cf7867-e2d2-4a12-93be-9371fe1a293c,churchlandlab,CSHL_010,F,2018-11-27,"left, left",C57BL/6J,Jax,3,None
3e97e1d3-2a0f-44e5-b63f-36196d78457a,churchlandlab,IBL_34,M,2018-05-24,None,None,Charles River,1,None
3f854f88-7879-4368-9e0d-41edea3bfab9,churchlandlab,IBL_11,M,2018-04-02,None,None,Charles River,1,None
4c9577b3-bdd5-4a15-b15b-5bc24a4439d3,churchlandlab,CSHL_002,M,2018-11-20,left ear,C57BL/6J,Jax,3,None
4f091667-90b9-4469-88f4-542c1ec07891,churchlandlab,CSHL_008,F,2018-11-27,left right,C57BL/6J,Jax,3,None
52a800fc-cbbc-45e9-97b1-ad6f6166e9af,churchlandlab,IBL_1,M,2018-04-02,None,None,Charles River,1,None
55381f61-4e47-4baa-beb9-70068c0ad62c,churchlandlab,IBL_46,M,2018-07-16,None,None,Charles River,3,None
5848e7f4-447d-42eb-9676-c914740df807,churchlandlab,CSHL_006,F,2018-11-27,R,C57BL/6J,Jax,3,None


## List subjects with some training summaries
>* current training protocol
>* date of the last session
>* current training status

First, let's find the last session for each subject, including the protocol used in the last session.

In [4]:
last_sessions = subjects.aggr(acquisition.Session & 'task_protocol!="NULL"', 
                              'task_protocol', session_start_time='max(session_start_time)')
last_sessions

subject_uuid,task_protocol,session_start_time calculated attribute
0936a724-2151-446f-98b2-92c3e9dd36dc,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-05-28 16:08:38
1208c089-8b8e-4a87-98f0-05a68fb18370,_iblrig_tasks_trainingChoiceWorld3.5.2,2019-05-28 12:35:50
19cf7867-e2d2-4a12-93be-9371fe1a293c,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-05-28 17:53:00
3e97e1d3-2a0f-44e5-b63f-36196d78457a,_iblrig_tasks_trainingChoiceWorld3.5.2,2019-05-28 15:33:10
3f854f88-7879-4368-9e0d-41edea3bfab9,_iblrig_tasks_trainingChoiceWorld3.5.2,2019-05-28 11:48:49
4c9577b3-bdd5-4a15-b15b-5bc24a4439d3,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-05-28 14:10:33
4f091667-90b9-4469-88f4-542c1ec07891,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-05-28 16:59:19
52a800fc-cbbc-45e9-97b1-ad6f6166e9af,_iblrig_tasks_trainingChoiceWorld3.5.2,2019-05-28 14:14:25
55381f61-4e47-4baa-beb9-70068c0ad62c,_iblrig_tasks_trainingChoiceWorld3.5.2,2019-05-28 16:51:04
5848e7f4-447d-42eb-9676-c914740df807,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-05-28 15:06:10


Second, get the current training status

In [5]:
last_sessions * behavior_analyses.SessionTrainingStatus

subject_uuid,session_start_time start time,task_protocol,training_status
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-05-28 16:08:38,_iblrig_tasks_habituationChoiceWorld3.7.4,ready for ephys
1208c089-8b8e-4a87-98f0-05a68fb18370,2019-05-28 12:35:50,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
19cf7867-e2d2-4a12-93be-9371fe1a293c,2019-05-28 17:53:00,_iblrig_tasks_habituationChoiceWorld3.7.4,ready for ephys
3e97e1d3-2a0f-44e5-b63f-36196d78457a,2019-05-28 15:33:10,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
3f854f88-7879-4368-9e0d-41edea3bfab9,2019-05-28 11:48:49,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
4f091667-90b9-4469-88f4-542c1ec07891,2019-05-28 16:59:19,_iblrig_tasks_habituationChoiceWorld3.7.4,ready for ephys
52a800fc-cbbc-45e9-97b1-ad6f6166e9af,2019-05-28 14:14:25,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
55381f61-4e47-4baa-beb9-70068c0ad62c,2019-05-28 16:51:04,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
5848e7f4-447d-42eb-9676-c914740df807,2019-05-28 15:06:10,_iblrig_tasks_habituationChoiceWorld3.7.4,over40days
7c751b49-55a6-4eac-9bdb-367faf2a18ee,2019-05-28 13:24:11,_iblrig_tasks_trainingChoiceWorld3.5.2,ready for ephys
